# Assignment 3 : Document Retrieval and Evaluation

---



In [2]:
#import all needed libraries here

import os
from bs4 import BeautifulSoup
import lxml
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import numpy as np
import pickle as pc
import math
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unrar x "/content/drive/MyDrive/FIRE_Dataset_EN_2010.rar" "/content/fire_dataset/"
# !tar -xvf "/content/fire_dataset/English-Data.tgz" -C "/content"
# !cp "/content/fire_dataset/queries.txt" "/content"
# !cp "/content/fire_dataset/relevance.txt" "/content"

import shutil
shutil.unpack_archive("/content/fire_dataset/FIRE_Dataset_EN_2010/English-Data.tgz", "/content/fire_dataset/FIRE_Dataset_EN_2010/")

qrels_path="/content/drive/MyDrive/en.qrels.76-125.2010.txt"
topics_path="/content/fire_dataset/FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt"



UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/MyDrive/FIRE_Dataset_EN_2010.rar

Creating    /content/fire_dataset                                     OK
Creating    /content/fire_dataset/FIRE_Dataset_EN_2010                OK
Extracting  /content/fire_dataset/FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt.gz       0%  OK 
Extracting  /content/fire_dataset/FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt       0%  OK 
Extracting  /content/fire_dataset/FIRE_Dataset_EN_2010/English-Data.tgz       3%  6%  9% 12% 15% 19% 22% 25% 28% 31% 34% 37% 41% 44% 47% 50% 53% 56% 60% 63% 66% 69% 72% 75% 78% 82% 85% 88% 91% 94% 97% 99%  OK 
All OK


# Function to read all docs 


In [19]:
def read_doc(path):
  global cnt
  global docs
  global docs_list
  for file in os.listdir(path):
    if os.path.isdir(os.path.join(path,file)):
      read_doc(os.path.join(path,file))
    else:
      if file.find("index") == -1 and cnt<3:
        fi = open(os.path.join(path,file),'r')
        bs = BeautifulSoup(fi.read())
        doc_id = bs.find("docno").text
        global i 
        i += 1
        index[doc_id] = i
        docs_list.append(doc_id)
        docs.append(bs.find("text").text[30:60])
        cnt+=1

##calling read_doc function to read whole corpus

In [20]:
global i 
cnt=0
index = {}
i = -1
docs = []
q_id = []
docs_list = []
read_doc("/content/fire_dataset/FIRE_Dataset_EN_2010/TELEGRAPH_UTF8")
fi = open(topics_path,'r')
data = fi.read()
bs = BeautifulSoup(data)
q_id = [int(id.text) for id in bs.find_all("num")]   # getting query id
desc = [desc.text[5:20] for desc in bs.find_all("desc")]  # getting description of query
 
q_id=q_id[:2]
desc=desc[:2]
docs.extend(desc)
 
for id in q_id:
  i += 1
  index[id] = i


print(docs_list)
print(docs)
print(index)
print(q_id)
print(desc)

['1060625_foreign_story_6397784.utf8', '1060209_foreign_story_5823850.utf8', '1060404_foreign_story_6054544.utf8']
['International\n\n Prachanda\n\n Ka', 'International\n\n Double standar', 'International\n\n Fine Print\n\n N', 'ons behind the ', 'cks by Hezbolla']
{'1060625_foreign_story_6397784.utf8': 0, '1060209_foreign_story_5823850.utf8': 1, '1060404_foreign_story_6054544.utf8': 2, 76: 3, 77: 4}
[76, 77]
['ons behind the ', 'cks by Hezbolla']


# Class for preprocessing data

In [21]:
class preprocessing:
  def __init__(self):
    self.docs_tokenized = [] # stores tokens of each doc
    self.words = set()
  def preprocess(self,docs):
    temp = []
    for doc in docs:
      doc = re.sub('(\d|[^\w\s])','',doc) #removing punctuations and digits
      doc = nltk.word_tokenize(doc.lower()) #tokenizing
      sw = set(stopwords.words('english'))
      doc = [token for token in doc  if token not in sw]# removing stop words
      lemmatizer = WordNetLemmatizer()
      doc = [lemmatizer.lemmatize(x) for x in doc] #lemmitizing
      self.docs_tokenized.append(doc)
      temp.extend(doc)
    self.words = set(temp)
    print(self.words)
    print(self.docs_tokenized)

In [22]:
ob1 = preprocessing() 
ob1.preprocess(docs)

{'print', 'ons', 'ka', 'international', 'standar', 'hezbolla', 'double', 'behind', 'fine', 'prachanda', 'cks', 'n'}
[['international', 'prachanda', 'ka'], ['international', 'double', 'standar'], ['international', 'fine', 'print', 'n'], ['ons', 'behind'], ['cks', 'hezbolla']]


# **Creating list of dictionaries where each dictionary contents word count of that document**

In [23]:
docs_dict = []    
for doc in ob1.docs_tokenized:
  temp_dict = {}
  for word in doc:
    temp_dict[word]=temp_dict.get(word,0)+1  #counting the frequency of each word
  docs_dict.append(temp_dict) 

print(docs_dict)   

[{'international': 1, 'prachanda': 1, 'ka': 1}, {'international': 1, 'double': 1, 'standar': 1}, {'international': 1, 'fine': 1, 'print': 1, 'n': 1}, {'ons': 1, 'behind': 1}, {'cks': 1, 'hezbolla': 1}]


#**Creating document frequency of each word which means how many documents have that word.**

In [24]:
df = {} #dictionary to store df
for doc in ob1.docs_tokenized:
  for word in set(doc):
    df[word] = df.get(word,0)+1  #Counting numbers of documents which contains specific word

print(df)


{'prachanda': 1, 'ka': 1, 'international': 3, 'double': 1, 'standar': 1, 'fine': 1, 'print': 1, 'n': 1, 'behind': 1, 'ons': 1, 'cks': 1, 'hezbolla': 1}


#**Calculating inverse document frequency-IDF for each word**

In [25]:
idf = {}
N = len(docs_dict)
for word in df:
  idf[word] = N/df[word]

print(idf)

{'prachanda': 5.0, 'ka': 5.0, 'international': 1.6666666666666667, 'double': 5.0, 'standar': 5.0, 'fine': 5.0, 'print': 5.0, 'n': 5.0, 'behind': 5.0, 'ons': 5.0, 'cks': 5.0, 'hezbolla': 5.0}


# **Class for finding tfidf vector**

In [26]:
class tf_idf:
  def __init__(self,doc_index,idf,docs_dict,words):
    self.doc_index=doc_index
    self.idf = idf
    self.docs_dict = docs_dict
    self.words = words
  """It returns a dictionary for document which contains word as key and it's Tf-idf score as value """  
  def getvectordict(self,doc_id):
    vector = {}
    doc = self.docs_dict[self.doc_index[doc_id]]  #Getting doc from docs dictionary with help of index
    n = sum(doc.values())
    if n!= 0:
      for word in doc:
        tf = doc[word] / n                        #Calculating tf
        tfidf = tf*self.idf[word]                 #Calculating tf-idf
        if tfidf!=0:
         vector[word] = tfidf
    print(vector)
    return vector   


In [27]:
ob2 = tf_idf(index,idf,docs_dict,ob1.words)

#**Find tfidf vectors of queries**

In [28]:
queries_tfidf = {}
for q in q_id:
    queries_tfidf[q] = ob2.getvectordict(q)

{'ons': 2.5, 'behind': 2.5}
{'cks': 2.5, 'hezbolla': 2.5}


#**Find tfidf vectors of documnets**

In [29]:
docs_tfidf = {}
for doc in docs_list:
    docs_tfidf[doc] = ob2.getvectordict(doc)

{'international': 0.5555555555555556, 'prachanda': 1.6666666666666665, 'ka': 1.6666666666666665}
{'international': 0.5555555555555556, 'double': 1.6666666666666665, 'standar': 1.6666666666666665}
{'international': 0.4166666666666667, 'fine': 1.25, 'print': 1.25, 'n': 1.25}


# **Calculating cosine similarity**

In [36]:
from numpy.linalg import norm
def c_s(a,b):
  dot_pr = 0
  print("a:",a," b:",b)
  av = list(a.values())
  bv = list(b.values())
  for word in a:
    #if word in b:
    dot_pr = dot_pr + 1#(a[word] * b[word])
  return (dot_pr/(norm(av)*norm(bv)))

In [37]:
cs = {}
for q in q_id:                    # iterating through each query
  qv = queries_tfidf[q]           # getting tfidf of specific query
  temp = {}
  for doc in docs_list:           # calculating cosine similarity of query and document
    dv = docs_tfidf[doc]    
    similarity = c_s(qv,dv)
    if  similarity != 0:               
     temp[doc] = similarity          
  cs[q] = temp
  print(cs[q])

a: {'ons': 2.5, 'behind': 2.5}  b: {'international': 0.5555555555555556, 'prachanda': 1.6666666666666665, 'ka': 1.6666666666666665}
a: {'ons': 2.5, 'behind': 2.5}  b: {'international': 0.5555555555555556, 'double': 1.6666666666666665, 'standar': 1.6666666666666665}
a: {'ons': 2.5, 'behind': 2.5}  b: {'international': 0.4166666666666667, 'fine': 1.25, 'print': 1.25, 'n': 1.25}
{'1060625_foreign_story_6397784.utf8': 0.23359884642829806, '1060209_foreign_story_5823850.utf8': 0.23359884642829806, '1060404_foreign_story_6054544.utf8': 0.2565707922359274}
a: {'cks': 2.5, 'hezbolla': 2.5}  b: {'international': 0.5555555555555556, 'prachanda': 1.6666666666666665, 'ka': 1.6666666666666665}
a: {'cks': 2.5, 'hezbolla': 2.5}  b: {'international': 0.5555555555555556, 'double': 1.6666666666666665, 'standar': 1.6666666666666665}
a: {'cks': 2.5, 'hezbolla': 2.5}  b: {'international': 0.4166666666666667, 'fine': 1.25, 'print': 1.25, 'n': 1.25}
{'1060625_foreign_story_6397784.utf8': 0.23359884642829806,

#**Getting top 10 documents for each query on basis of there cosine similarity**

In [41]:
from heapq import nlargest
rank_list = {}
for q in q_id:
  rank_list[q] = nlargest(5,cs[q],key = cs[q].get)
  print(cs[q].get)

print(rank_list)

<built-in method get of dict object at 0x7fe70bd1a0d8>
<built-in method get of dict object at 0x7fe70f730bd0>
{76: ['1060404_foreign_story_6054544.utf8', '1060625_foreign_story_6397784.utf8', '1060209_foreign_story_5823850.utf8'], 77: ['1060404_foreign_story_6054544.utf8', '1060625_foreign_story_6397784.utf8', '1060209_foreign_story_5823850.utf8']}


# **Reading qrels**

In [43]:
fr = open(qrels_path,'r')
qr = {}
for line in fr.readlines():
  data = line.split()
  if data[0] not in qr:
   qr[data[0]] = {}
   qr[data[0]][data[2]] = int(data[3])          
  else: 
    qr[data[0]][data[2]] = int(data[3])

qr= {'76': {'1060404_foreign_story_6054544.utf8': 1,'1060625_foreign_story_6397784.utf8': 0,'1060209_foreign_story_5823850.utf8': 1},'77':{'1060404_foreign_story_6054544.utf8': 0,'1060625_foreign_story_6397784.utf8': 1,'1060209_foreign_story_5823850.utf8':1 } }
print(qr)

{'76': {'1060404_foreign_story_6054544.utf8': 1, '1060625_foreign_story_6397784.utf8': 0, '1060209_foreign_story_5823850.utf8': 1}, '77': {'1060404_foreign_story_6054544.utf8': 0, '1060625_foreign_story_6397784.utf8': 1, '1060209_foreign_story_5823850.utf8': 1}}


# **Calculating Mean Average Precision**

In [53]:
avg_precision = []                                
for q in q_id:                                             #Iterating through each query
  rk = rank_list[q]                                        # Getting rank list of specific query
  relevance = 0                                            # Store number of relevant documents found
  count = 0                                                #To store count of docs
  precision_sum = 0 
  for doc in rk:
    print("doc:",doc)
    count += 1                         
    if doc in qr[str(q)]:
      if qr[str(q)][doc] == 1:                             #Checking if it's relavant document 
        print('qr[',q,'][',doc,']')
        relevance += 1                          
        precision_sum += (relevance/count)                 #Calculating precision and adding it to local precision_sum                    
        print('relevence:',relevance,'cnt:',count)
  if relevance==0:
    avg_precision.append(0)     
  else:
    avg_precision.append(precision_sum/relevance)
print(avg_precision)
map = sum(avg_precision)/len(avg_precision)      

print("MAP : ",round(map,2))

doc: 1060404_foreign_story_6054544.utf8
qr[ 76 ][ 1060404_foreign_story_6054544.utf8 ]
relevence: 1 cnt: 1
doc: 1060625_foreign_story_6397784.utf8
doc: 1060209_foreign_story_5823850.utf8
qr[ 76 ][ 1060209_foreign_story_5823850.utf8 ]
relevence: 2 cnt: 3
doc: 1060404_foreign_story_6054544.utf8
doc: 1060625_foreign_story_6397784.utf8
qr[ 77 ][ 1060625_foreign_story_6397784.utf8 ]
relevence: 1 cnt: 2
doc: 1060209_foreign_story_5823850.utf8
qr[ 77 ][ 1060209_foreign_story_5823850.utf8 ]
relevence: 2 cnt: 3
[0.8333333333333333, 0.5833333333333333]
MAP :  0.71


In [49]:
d={'a':4,'5':6}
d2={'5':{'a':4,'5':6},'c':{'a':4,'5':6}}
for ele in d:
    print(ele)
    if ele in d2:
        print("yes")
    else:
        print("no")

a
no
5
yes
